## Zadanie - Grupowanie zwierząt

Wykonaj analizę skupień dla zbioru danych `zoo`. Plik z danymi znajduje się w katalogu repozytorium [dane/zoo_nc.csv](dane/zoo_nc.csv) i jest zgodny z formatem CSV. Pierwszy wiersz pliku zawiera nazwy zmiennych.

Dane zawierają 101 przypadków zwierząt opisanych 17 zmiennymi:
- pierwsza kolumna `animal_name` to nazwa zwierzaka
- 15 zmiennych ma wartości binarne oznaczające występowanie danej cechy u zwierzęcia, np. `hair`, `feathers`, `eggs`, `milk`, itd. 
- zmienna `legs` określa ilość nóg i posiada wartości od 0 do 8

Przeprowadź analizę skupień zbioru danych znajdując istotnie różniące sie grupy zwierząt. Spodziewamy się, że wyodrębnione grupy będą odzwierciedlały podział zwierząt na gromady: ssaki, gady, płazy, ptaki, ...

1. Wczytaj dane i przygotuj zbiór uczący według poniższych instrukcji:
  - zmienna `animal_name` zawiera nazwy zwierząt, unikatowe dla kazdego przypadku. Usuń tą zmienną ze zbioru uczącego.
  - zmienna `legs` określa liczbę nóg i przyjmuje wartości całkowite z zakresu od 0 do 8, których zakres znacznie odbiega od zakresu pozostałych zmiennych o wartościach binarnych. Zamień zmienną kategoryczną `legs` za pomocą kodowania `one-hot` na zestaw zmiennych binarnych. Przydatna może być funkcja [pandas.get_dummies](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) lub transfromer danych [sklearn.preprocessing.OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)
  - usuń wartości brakujące jeśli istnieją.

Wynikowy zbiór uczący powinien zawierać wyłącznie wartości binarne $\{0,1\}$ we wszystkich zmiennych .  

In [2]:
import pandas as pd

df = pd.read_csv('dane/zoo_nc.csv')
df = df.drop('animal_name', axis=1)
df = pd.get_dummies(df, columns=['legs'], prefix='legs')
df = df.dropna()
df = df.astype(int)

print(df)

     hair  feathers  eggs  milk  airborne  aquatic  predator  toothed  \
0       1         0     0     1         0        0         1        1   
1       1         0     0     1         0        0         0        1   
2       0         0     1     0         0        1         1        1   
3       1         0     0     1         0        0         1        1   
4       1         0     0     1         0        0         1        1   
..    ...       ...   ...   ...       ...      ...       ...      ...   
96      1         0     1     0         1        0         0        0   
97      1         0     0     1         0        0         1        1   
98      0         0     1     0         0        0         0        0   
99      0         1     1     0         1        0         0        0   
100     1         0     0     1         0        0         1        1   

     backbone  breathes  ...  fins  tail  domestic  catsize  legs_0  legs_2  \
0           1         1  ...     0     0    

2. Wykonaj analizę skupień w celu znalezienia naturalnych grup w zbiorze zwierząt za pomocą *JEDNEJ* z poniższych metod:
   1. Metoda k-średnich - wyznacz optymalną liczbę klastrów za pomocą metody łokcia 
   2. Grupowanie hierarchiczne metodą łączenia zupełnego lub metodą połączeń średnich. Wyświetl dendrogram i na jego podstawie wybierz liczbę skupisk. Przypisz zwierzęta ze zbioru uczącego do tych skupisk.  
   3. Metoda DBSCAN - zastosuj metrykę `manhattan`, `cityblock` lub `hamming`. W przypadku danych binarnych metryki te obliczają liczbę różniących się bitów. Dobierz promień `eps` tak aby uzyskać nie mniej niż 2 i nie więcej niż 7 grup zwierząt przy jak najmniejszej liczbie szumu. Minimalna liczebność grup nie powinna być mniejsza niż 5.

In [3]:
from sklearn.cluster import KMeans

inertias = []
k_range = range(1, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(df)
    inertias.append(kmeans.inertia_)


optimal_k = 4
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
clusters = kmeans.fit_predict(df)

df['cluster'] = clusters
print(f"liczba klastrów: {optimal_k}")
print(df['cluster'].value_counts().sort_index())

liczba klastrów: 4
cluster
0    31
1     8
2    39
3    23
Name: count, dtype: int64


3. Dla każdej grupy zwierząt uzyskanej w wyniku analizy skupień z poprzediego punktu wyznacz:
   * liczbę zwierząt w grupie 
   * nazwy zwierząt w grupie
   * procentową zawartość ssaków w grupie. Informację o tym, do jakiej gromady należą zwierzęta odczytasz z pliku [dane/zoo_classes.csv](dane/zoo_classes.csv). Zmienna ``class_type`` przybiera dla ssaków wartość 1, zaś zmienna ``class_name`` zawiera nazwę gromady i w przypadku ssaków posiada wartość ``Mammal``  

In [ ]:
original_df = pd.read_csv('dane/zoo_nc.csv')
classes_df = pd.read_csv('dane/zoo_classes.csv')

df_with_names = df.copy()
df_with_names['animal_name'] = original_df['animal_name']

merged_df = df_with_names.merge(classes_df, on='animal_name', how='left')

for cluster_id in sorted(df['cluster'].unique()):
    cluster_data = merged_df[merged_df['cluster'] == cluster_id]
    
    count = len(cluster_data)
    animals = cluster_data['animal_name'].tolist()
    mammals = cluster_data[cluster_data['class_type'] == 1]
    mammal_percentage = (len(mammals) / count) * 100 if count > 0 else 0
    
    print(f"Grupa {cluster_id}:")
    print(f"Liczba zwierząt: {count}")
    print(f"Nazwy zwierząt: {animals}")
    print(f"Procent ssaków: {mammal_percentage:.1f}%")


Grupa 0:
Liczba zwierząt: 33
Nazwy zwierząt: ['bass', 'carp', 'catfish', 'chub', 'clam', 'crab', 'crayfish', 'dogfish', 'dolphin', 'frog', 'frog', 'frog', 'frog', 'haddock', 'herring', 'lobster', 'newt', 'pike', 'piranha', 'pitviper', 'porpoise', 'seahorse', 'seasnake', 'seawasp', 'slowworm', 'slug', 'sole', 'starfish', 'stingray', 'toad', 'tuatara', 'tuna', 'worm']
Procent ssaków: 6.1%

Grupa 1:
Liczba zwierząt: 23
Nazwy zwierząt: ['chicken', 'crow', 'dove', 'duck', 'flamingo', 'flea', 'gnat', 'gull', 'hawk', 'honeybee', 'housefly', 'ladybird', 'lark', 'moth', 'parakeet', 'pheasant', 'skimmer', 'skua', 'sparrow', 'swan', 'termite', 'wasp', 'wren']
Procent ssaków: 0.0%

Grupa 2:
Liczba zwierząt: 39
Nazwy zwierząt: ['aardvark', 'antelope', 'bear', 'boar', 'buffalo', 'calf', 'cavy', 'cheetah', 'deer', 'elephant', 'fruitbat', 'giraffe', 'goat', 'gorilla', 'hamster', 'hare', 'leopard', 'lion', 'lynx', 'mink', 'mole', 'mongoose', 'opossum', 'oryx', 'platypus', 'polecat', 'pony', 'puma', 'pu

4. Sprawdź, do której grupy przypisany został ostatni, tajemniczy zwierz o nazwie `X`. Odpowiedz na pytanie, czy jest on ssakiem?

In [8]:
mystery_animal = merged_df[merged_df['animal_name'] == 'X']

cluster_id = mystery_animal['cluster'].iloc[0]
class_type = mystery_animal['class_type'].iloc[0]
is_mammal = class_type == 1
    
print(f"Zwierzę X zostało przypisane do grupy: {cluster_id}")
print(f"czy jest ssakiem: {'tak' if is_mammal else 'nnie'}")


Zwierzę X zostało przypisane do grupy: 2
czy jest ssakiem: nnie
